In [1]:
import pandas as pd
import requests

# Get data from API

## Check dates of original dataset

In [2]:
df_orig = pd.read_csv('../project_data/complaints_1.csv')

In [9]:
type(df_orig['Date received'][0])

str

Dates are in string format. Convert to datetime.

In [18]:
df_orig.rename(columns={'Date received': 'date'}, inplace=True)

df_orig['date'] = pd.to_datetime(df_orig['date'])

In [21]:
min(df_orig['date'])

Timestamp('2020-03-17 00:00:00')

In [22]:
max(df_orig['date'])

Timestamp('2021-03-01 00:00:00')

The most recent was 2021-03-01

## Use API

Submit GET request with parameters

In [23]:
url = 'https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/'

parameters = {'date_received_min': '2021-03-02',
              'has_narrative': True
}
r = requests.get(url, params=parameters)

Make data into a dictionary format

In [24]:
data = r.json()

In [26]:
# Instatiate empty dictionary
new_dict = {}

# Create empty lists to gather data
class_list = []
narrative_list = [] 

# Loop through data and add to dictionary
for i in range(len(data['hits']['hits'])):
    class_list.append(data['hits']['hits'][i]['_source']['product'])
    narrative_list.append(data['hits']['hits'][i]['_source']['complaint_what_happened'])
new_dict['class'] = class_list
new_dict['narrative'] = narrative_list

In [29]:
df = pd.DataFrame.from_dict(new_dict)
df

,class,narrative
0,"Credit reporting, credit repair services, or o...",I have determined that there are some creditor...
1,"Credit reporting, credit repair services, or o...","On XX/XX/2020, I initially bought some great g..."
2,"Credit reporting, credit repair services, or o...",I sent a letter plus I did a dispute but they ...
3,Checking or savings account,Wells Fargo failed to provide outstanding reli...
4,"Credit reporting, credit repair services, or o...",Wells Fargo continuously reports a late paymen...
5,Checking or savings account,Bank account open that i have no knowledge of ...
6,"Credit reporting, credit repair services, or o...",I signed up for PayPal Credit in XXXX and was ...
7,Mortgage,CFPB : FILING AN OFFICIAL COMPLAINT REGARDING ...
8,"Credit reporting, credit repair services, or o...",XXXX account was paid after it was sent into c...
9,"Credit reporting, credit repair services, or o...",I did not authorize anyone employed by these c...
